## Problema 1

Suponemos que $n$ es la dimensión de los datos. Explicar por qué el método de *sliced Score Matching*
propuesto en el artículo necesita 1 solo paso de *backpropagation* en lugar de $n$ como el algoritmo
original de *score matching*.

**Respuesta**

El método de *score matching* consiste en estimar una función de *score*:
$$
s_d(x) = \nabla_x \log p_d(x)
$$

Donde $p_d(x)$ es la distribución de los datos.

Para encontrar un estimador del *score*, se busca una función paramétrica $s_m(x; \theta)$ que minimice la divergencia de Fisher:

$$
    L = \frac{1}{2} E_{p_d}\left[s_m(x; \theta) -s_d(x) \right]
$$

En el algoritmo original de *Score Matching*, se trata de minimizar una función que, según [1], sale de la integración por partes de $L$. Esto es, se muestra que $L$ puede ser escrita como $L = J + C$. Donde $C$ una constante de integración (que no puede minimizarse) y $J$ es:

$$
    J = E\left[tr(\nabla_x s_m(x; \theta)) +\frac{1}{2}||s_m(x;\theta)||_2^2\right]
$$

Cuyo estimador no sesgado usado es:
$$
    \hat J = \frac{1}{N} \sum_{i=1}^N \left[ tr(\nabla_x s_m(x_i; \theta)) +\frac{1}{2}||s_m(x_i;\theta)||_2^2 \right]
$$


Donde $\nabla_x s_m(x; \theta)$ es una matriz hessiana, pues $s_m(x; \theta)$ ya implica una derivada de primer orden. Entonces, para calcular cada elemento de la diagonal de la hessiana, se requieren de $n$ operaciones de gradiente.

En contraste, En *sliced score matching* se usa la idea de que si el promedio de la diferencia proyectada de una función $h(x;\theta)$ y el *score* del conjunto de datos es pequeño,  entonces $h(x;\theta)$ debe estar cercana al *score*. De esta forma, en lugar de minimizar la divergencia de Fisher, se trata de minimizar:

$$
    L = \frac{1}{2}E_{p_v}E_{p_d} \left[(v^T s_m(x; \theta) - v^Ts_d(x))^2 \right]
$$

Donde $v$ es una dirección aleatoria que debe cumplir con requisitos bastante comúnes. De misma forma, se encuentra que $L = J + C$. Sin embargo, un estimador no sesgado de este $J$ es bastante más sencillo:

$$
    \hat J =  \frac{1}{N} \frac{1}{M} \sum_{i=1}^N \sum_{j=1}^M v_{ij}^T \nabla_x s_m(x_i; \theta) v_{ij} + \frac{1}{2} (v_{ij}^T s_m(x; \theta))^2
$$

Con $v^T\nabla_x s_m(x; \theta) = \text{grad}(s_m(x; \theta), \theta)$

Esto es, en lugar de realizar una operación de gradiente por cada dimensión para calcular la diagonal de la hessiana, se realizan tan sólo $M$ operaciones de gradiente. En el trabajo de *Sliced Score Matching* ([2]), demostraron que la estimación de los parámetros $\theta$ de la distribución de parámetros es consistente para toda $M \in N^+$.

## Problema 2

Implementa el método de *sliced score matching* en PyTorch para generar imágenes a partir de los datos de MNIST.

In [1]:
import torch

In [7]:
x = torch.rand(3, 4)

In [8]:
x

tensor([[0.1727, 0.5285, 0.5054, 0.1514],
        [0.6204, 0.7298, 0.0988, 0.4478],
        [0.8685, 0.3192, 0.0027, 0.3779]])

In [10]:
x.sum(dim=-1)

tensor([1.3581, 1.8969, 1.5682])

In [82]:
directions = torch.randn(2, 3, 4)

In [83]:
directions

tensor([[[-1.5555,  0.4553,  0.4646,  0.1418],
         [-0.5424, -0.2754,  2.1603, -0.5904],
         [ 1.4229, -2.8829, -1.1161,  1.4301]],

        [[ 0.6903,  1.8170,  0.6446, -0.3864],
         [-0.2109,  1.3489, -0.3616, -0.8607],
         [ 0.1221, -0.1169,  0.9229,  2.4878]]])

In [84]:
directions.shape

torch.Size([2, 3, 4])

In [91]:
torch.matmul(directions.view(2, 3, 1, 4), x.view(1, 3, 4, 1)).shape

torch.Size([2, 3, 1, 1])

tensor([[[[-1.5555,  0.4553,  0.4646,  0.1418]],

         [[-0.5424, -0.2754,  2.1603, -0.5904]],

         [[ 1.4229, -2.8829, -1.1161,  1.4301]]],


        [[[ 0.6903,  1.8170,  0.6446, -0.3864]],

         [[-0.2109,  1.3489, -0.3616, -0.8607]],

         [[ 0.1221, -0.1169,  0.9229,  2.4878]]]])